# Assignment 5: "No one can be told what the Ranktrix is. You have to see it for yourself."

## © Cristian Danescu-Niculescu-Mizil 2023

## CS/INFO 4300 Language and Information

### Due by 11:59PM on Friday March 03, 2023

**Instructions**

For code completion tasks, just type your code after the comment marking the place.  For questions, use as many notebook cells as needed to compute intermediate stuff.

You are strongly encouraged to write sensible **test cases** for your code.

This is an **individual** assignment.

If you use any outside sources (e.g. research papers, StackOverflow) please list your sources.

In this assignment we will explore evaluation of an information retrevial system where both queries and results are movies. Ever wanted to know what the most similar movie to "The Matrix" is, in terms of language? Now is your chance! You take the blue pill - the story ends, you wake up in your dorm on west campus and believe whatever you want to believe. You take the red pill - you stay in CS/INFO 4300 and we show you how deep the rabbit-hole goes.

**Guidelines**

* All cells that contain the blocks that read `# YOUR CODE HERE` are editable and are to be completed to ensure you pass the test-cases. Make sure to write your code where indicated.

* All cells that read `YOUR ANSWER HERE` are free-response cells that are editable and are to be completed.

* Please delete raise `NotImplementedError()` after filling in the function code. It is only meant to be a temporary placeholder

* You may use any number of notebook cells to explore the data and test out your functions, although you will only be graded on the solution itself.

* You are unable to modify the read-only cells.

* You should also use Markdown cells to explain your code and discuss your results when necessary.
Instructions can be found [here](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html).

* All floating point values should be printed with **2 decimal places** precision. You can do so using the built-in round function.

* **Never delete any code / free response and autograder test cell.**

* Do not delete the cells for optional questions, even if you do not choose to answer them.

**Learning Objectives**

This project aims to help you get comfortable working with the following tools / technologies / concepts:

* TF-IDF vectorization using sklearn
* Similarity matrices
* Precision & Recall
* Cosine similarity vs Jaccard similarity
* Rocchio Algorithm

**Grading**

For code-completion questions you will be graded on passing the public test cases we have included, as well as any hidden test cases that we have supplemented to ensure that your logic is correct.

For free-response questions you will be manually graded on the quality of your answer.

## Introduction
In this assignment, we will be building a system that allows you to query for movies similar to a given movie. Unlike before, queries and information retrieved have the same type -- i.e. movies are *both* queries and results. To accomplish this task, we will utilize a dataset of movies and their transcripts. We will begin by using the language contained in the transcripts to do basic queries. We will continue to use the vector space model, encoding "documents" (here, a document is a movie script) as tf-idf vectors.

In [ ]:
from __future__ import print_function
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from numpy import linalg as LA
import json
import math
%matplotlib inline
import matplotlib.pyplot as plt

## Using sklearn to get tf-idf vectors
On the last assignment, we used an inverted index to quickly compute queries by taking advantage of the sparsity of tf-idf vectors in our vector space. However, the dataset we are considering for this assignment is small enough such that we can use explicit vectors, rather than an inverted index, to compute cosine similarities. We also will not be implementing tf-idf by hand -- we will be using an existing implementation from the library [sklearn,](http://scikit-learn.org/stable/) which provides a lot of good implementations of machine learning algorithms in python. We will be making heavy use of this powerful library later in the semester, so using it to extract tfidf features is a good starting point!

In [ ]:
with open("movie_scripts_data.json") as f:
    data = json.loads(f.readlines()[0])
num_movies = len(data)
print("Loaded {} movie transcripts".format(num_movies))
print("Each movie transcript is a dictionary with the following keys...")
print(data[0].keys())

# Here, we will assign an index for each movie_id. This index will help us access data in numpy matrices.
movie_id_to_index = {movie_id:index for index, movie_id in enumerate([d['movie_id'] for d in data])}

# We will also need a dictionary maping movie names to movie ids
movie_name_to_id = {name:mid for name, mid in zip([d['movie_name'] for d in data],
                                                     [d['movie_id'] for d in data])}
movie_id_to_name = {v:k for k,v in movie_name_to_id.items()}

# and because it might be useful...
movie_name_to_index = {name:movie_id_to_index[movie_name_to_id[name]] for name in [d['movie_name'] for d in data]}
movie_index_to_name = {v:k for k,v in movie_name_to_index.items()}

movie_names = [name for name in [d['movie_name'] for d in data]]

print("The index of \"{}\" is {}".format(data[7]['movie_name'], movie_id_to_index[data[7]['movie_id']]))

We can see that each movie is assigned an "index" (from 0 to 616). These will correspond to the rows of a document-by-tfidf score matrix.

## Question 1 (Code Completion): TFIDF Vectorizer
Read up on sklearn's [tfidf-vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). This function takes in a list of documents and some parameters related to parsing and outputs a document-by-vocabulary matrix, where entry i,j corresponds to the tfidf score of word j in document i. 

Your first job is to make a TfidfVectorizer object that includes the following preprocessing properties (look at the documentation to learn how to achieve these): 
- It only considers words that appear in _at least_ ten documents, but in _no more_ than 80% of all documents.
- It computes a maximum of 5000 features, and detects and filters out stopwords in English.
- It should normalize all tfidf vectors to have an l2 norm of 1.

Once you've made this object, call its `fit_transform()` function on the list of *scripts* (not titles) in data. This should produce a numpy matrix whose shape is the number of documents by the number of words you're considering (which has a maximum of 5000).

In [ ]:
n_feats = 5000
doc_by_vocab = np.empty([len(data), n_feats])

def build_vectorizer(max_features, stop_words, max_df=0.8, min_df=10, norm='l2'):
    """Returns a TfidfVectorizer object with the above preprocessing properties.
    
    Note: This function may log a deprecation warning. This is normal, and you
    can simply ignore it.
    
    Parameters
    ----------
    max_features : int
        Corresponds to 'max_features' parameter of the sklearn TfidfVectorizer 
        constructer.
    stop_words : str
        Corresponds to 'stop_words' parameter of the sklearn TfidfVectorizer constructer. 
    max_df : float
        Corresponds to 'max_df' parameter of the sklearn TfidfVectorizer constructer. 
    min_df : float
        Corresponds to 'min_df' parameter of the sklearn TfidfVectorizer constructer. 
    norm : str
        Corresponds to 'norm' parameter of the sklearn TfidfVectorizer constructer. 

    Returns
    -------
    TfidfVectorizer
        A TfidfVectorizer object with the given parameters as its preprocessing properties.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
tfidf_vec = build_vectorizer(n_feats, "english")
doc_by_vocab = tfidf_vec.fit_transform([d['script'] for d in data]).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that build_vectorizer returns the correct output"""
assert type(doc_by_vocab) == np.ndarray
assert type(tfidf_vec) == TfidfVectorizer
assert sum(doc_by_vocab[2,:]) < 20
assert doc_by_vocab.shape == (617, 5000)
assert 'zoo' in index_to_vocab.values()

## Question 2 (Code Completion): Cosine Similarity
You will implement the function below which takes in the names of two movies, a term-document matrix of movie transcripts, and a dictionary that maps movie names to the corresponding row index in the term-document matrix.

Remember that cosine similarity is defined as:

$$ cossim(\vec{q}, \vec{d_j}) = \frac{\vec{q} \cdot \vec{d_j}}{\|\vec{q}\| \cdot \|\vec{d_j}\|}$$


Hint: As always, make good use of numpy to make your implementation efficient, as this method will be called in later questions. However, note that you cannot import off-the-shelf implementation of cosine similarity for this question.

In [ ]:
def get_sim(mov1, mov2, input_doc_mat, input_movie_name_to_index):
    """Returns a float giving the cosine similarity of 
       the two movie transcripts.
    
    Params: {mov1 (str): Name of the first movie.
             mov2 (str): Name of the second movie.
             input_doc_mat (numpy.ndarray): Term-document matrix of movie transcripts, where 
                    each row represents a document (movie transcript) and each column represents a term.
             movie_name_to_index (dict): Dictionary that maps movie names to the corresponding row index 
                    in the term-document matrix.}
    Returns: Float (Cosine similarity of the two movie transcripts.)
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that get_sim returns the correct output"""
print("Similarity: Star Wars vs. Jurassic Park")
print("======")
test1 = get_sim('star wars', 'jurassic park', doc_by_vocab, movie_name_to_index)
print(test1)
assert test1 < 0.07 and test1 > 0.06
print("")

print("Similarity: Star Wars vs. Star Trek: Generations")
print("======")
test2 = get_sim('star wars', 'star trek: generations', doc_by_vocab, movie_name_to_index)
print(test2)
assert test2 < 0.25 and test2 > 0.20


## Question 3 (Code Completion): Term Similarity
Complete the function `top_terms`, which takes in the list of movie names, and returns the top matching tfidf terms from these movie transcripts. Consider doing an element-wise product of the movies' tfidf vectors. If all vectors have a high value for a particular term, then it is contributing to the cosine similarity for each (e.g. if both "star wars" and "jurassic park" both had high tfidf weights for "fight," then this word would likely be a top_term for the two). After performing the element-wise product, find the indices that produce the highest values.

In [ ]:
def top_terms(movs, input_doc_mat, index_to_vocab, movie_name_to_index, top_k=10):
    """Returns a list of the top k similar terms (in order) between the
        inputted movie transcripts.
    
    Parameters
    ----------
    movs : str list (Length >= 2)
        List of movie names 
    input_doc_mat : np.ndarray
        The term document matrix of the movie transcripts. input_doc_mat[i][j] is the tfidf
        of the movie i for the word j.
    index_to_vocab : dict
         A dictionary linking the index of a word (Key: int) to the actual word (Value: str). 
         Ex: {0: 'word_0', 1: 'word_1', .......}
    movie_name_to_index : dict
         A dictionary linking the movie name (Key: str) to the movie index (Value: int). 
         Ex: {'movie_0': 0, 'movie_1': 1, .......}
    top_k : int
        The k in the top k similar words to be returned. Ex: If top_k = 8, return top 8 similar words

    Returns
    -------
    list
        A list of the top k similar terms (in order) between the inputted movie transcripts
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that get_sim returns the correct output"""
print("Top ten terms between: Star Wars and Jurassic Park")
print("======")
term_test_1 = top_terms(['star wars', 'jurassic park'], doc_by_vocab, index_to_vocab, movie_name_to_index)
for term in term_test_1:
    print(term)
assert 'force' in term_test_1

print("")
print("Top ten terms between: Star Wars and Star Trek: Generations")
term_test_2 = top_terms(['star wars', 'star trek: generations'], doc_by_vocab,index_to_vocab, movie_name_to_index)
assert 'star' in term_test_2
print("======")
for term in term_test_2:
    print(term)
    
print("")
print("Top ten terms between: Star Wars and Star Trek: Generations and Jurassic Park")
term_test_3 = top_terms(['star wars', 'star trek: generations', 'jurassic park'], doc_by_vocab,index_to_vocab, movie_name_to_index)
assert 'attack' in term_test_3
print("======")
for term in term_test_3:
    print(term)


## Question 3b (Free Response): Common terms interpretation


In the cell below, state what you noticed about the common terms shared by the three movies in the test case ('star wars', 'star trek: generations', 'jurassic park'). Did the common terms represent some common themes, categories, or genres shared by these movies? 
<br>

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

## Question 4 (Code Completion): Similar Transcripts, Similar Movies w/ Cosine Sim

Given your `get_sim` function, you can now compute how similar movies are to one another! Here, we will first precompute the similarity between every possible pair of movies, and store it in a movies-by-movies matrix. Given this matrix, for a given movie, it is possible to produce a ranking of how similar all other movies are to the given movie. For instance, we will see what movies are the most similar and dissimilar to "star wars".

In [ ]:
def build_movie_sims_cos(n_mov, movie_index_to_name, input_doc_mat, movie_name_to_index, input_get_sim_method):
    """Returns a movie_sims matrix of size (num_movies,num_movies) where for (i,j):
        [i,j] should be the cosine similarity between the movie with index i and the movie with index j
        
    Note: You should set values on the diagonal to 1
    to indicate that all movies are trivially perfectly similar to themselves.
    
    Params: {n_mov: Integer, the number of movies
             movie_index_to_name: Dictionary, a dictionary that maps movie index to name
             input_doc_mat: Numpy Array, a numpy array that represents the document-term matrix
             movie_name_to_index: Dictionary, a dictionary that maps movie names to index
             input_get_sim_method: Function, a function to compute cosine similarity}
    Returns: Numpy Array 
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
movie_sims_cos = build_movie_sims_cos(num_movies, movie_index_to_name, doc_by_vocab, movie_name_to_index, get_sim)

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that build_movie_sims_cos returns the correct output"""
assert type(movie_sims_cos) == np.ndarray
assert movie_sims_cos.shape == (617,617)
assert movie_sims_cos[15,15] == 1.0
assert sum(movie_sims_cos[:,5]) > 40
test_star_trek = movie_sims_cos[movie_name_to_index["star trek iii: the search for spock"]][movie_name_to_index["star trek: the wrath of khan"]]
assert test_star_trek > 0.6 and test_star_trek < 0.7

## Question 4b (Code Completion): Similar Transcripts, Similar Movies w/ Jaccard Sim

For a baseline comparison, we will also consider the similarity between movies in terms of categories. Specifically, you'll notice that each movie is associated with a list of categories. One could ignore the transcripts *entirely* and say that the similarity between any two movies is the jaccard similarity of their category sets. While this is a very rough way to measure similarity, we will use it as a baseline for comparison with the linguistic methods.

Hint: The keys of the dictionary are printed out in the same cell the data is loaded in.

In [ ]:
def build_movie_sims_jac(n_mov, input_data):
    """Returns a movie_sims_jac matrix of size (num_movies,num_movies) where for (i,j) :
        [i,j] should be the jaccard similarity between the category sets for movies i and j
        such that movie_sims_jac[i,j] = movie_sims_jac[j,i]. 
        
    Note: 
        Movies sometimes contain *duplicate* categories! You should only count a category once
        
        A movie should have a jaccard similarity of 1.0 with itself.
    
    Params: {n_mov: Integer, the number of movies,
            input_data: List<Dictionary>, a list of dictionaries where each dictionary 
                     represents the movie_script_data including the script and the metadata of each movie script}
    Returns: Numpy Array 
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
movie_sims_jac = build_movie_sims_jac(num_movies,data)

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that build_movie_sims_cos returns the correct output"""
assert type(movie_sims_jac) == np.ndarray
assert movie_sims_jac.shape == (617,617)
assert sum(movie_sims_jac[:,5]) > 60 and sum(movie_sims_jac[:,5]) < 70

## Question 4c (Free Response): Similar Transcripts, Similar Movies

Using the `movie_sims_cos` and `movie_sims_jac` matrices you computed, we are now going to compare cosine vs. jaccard similarity by printing the 10 most similar (ignoring itself) and 10 most dissimilar movies to 'star wars'.

In [ ]:
def get_ranked_movies(mov, matrix):
    """
    Return sorted rankings (most to least similar) of movies as 
    a list of two-element tuples, where the first element is the 
    movie name and the second element is the similarity score
    
    Params: {mov: String,
             matrix: np.ndarray}
    Returns: List<Tuple>
    """
    
    # Get movie index from movie name
    mov_idx = movie_name_to_index[mov]
    
    # Get list of similarity scores for movie
    score_lst = matrix[mov_idx]
    mov_score_lst = [(movie_index_to_name[i], s) for i,s in enumerate(score_lst)]
    
    # Do not account for movie itself in ranking
    mov_score_lst = mov_score_lst[:mov_idx] + mov_score_lst[mov_idx+1:]
    
    # Sort rankings by score
    mov_score_lst = sorted(mov_score_lst, key=lambda x: -x[1])
    
    return mov_score_lst


def print_top(mov, matrix, sim_type, k=10):
    """
    Print the k most and least similar movies to 'star wars'
    
    Params: {mov: String,
             matrix: np.ndarray,
             sim_type: String,
             k: Integer}
    Returns: None
    """
    
    mov_score_lst = get_ranked_movies(mov, matrix)
    
    print("Top {} most similar movies to {} [{}]".format(k, 'star wars', sim_type))
    print("======")
    for (mov, score) in mov_score_lst[:k]:
        print("%.3f %s" % (score, mov))

    print()
    
    print("Top {} least similar movies to {} [{}]".format(k, 'star wars', sim_type))
    print("======")
    for (mov, score) in mov_score_lst[-k:][::-1]:
        print("%.3f %s" % (score, mov))

In [ ]:
print_top('star wars', movie_sims_cos, 'cosine sim')

In [ ]:
print_top('star wars', movie_sims_jac, 'jaccard')

In the cell below, analyze the most similar and least similar results above for both cosine similarity and jaccard similarity. Please comment on how well (or poorly) you think both of the similarity measures performed. Do these results make sense to you? Why are certain movies ranked more similar to Star Wars in one similarity measure than the other one?

<br>

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;"></div>

## Question 5 (Code Completion): Evaluating our rankings with precision-recall curves

Given that we are able to produce a most-to-least similar ranking of all other movies given all other movies, we can now ask the question: "How good are our rankings?"

For this part, we will be using the following (query, [related movie list]) pairs for you to evaluate against. We will treat these data as ground truth. More generally, you could imagine that these "ground truths" result from aggregated user feedback from a movie recomendation system like Netflix.

In [ ]:
queries = [('the matrix', ['the bourne supremacy',
                           'being john malkovich',
                           'smoke',
                           'erin brockovich',
                           'an officer and a gentleman',
                           'smokin\' aces',
                           'pitch black',
                           'out of sight',
                           'clerks.',
                           'xxx',
                           'the x files',
                          ]),
           ('star wars',  ['star wars: the empire strikes back',
                            'star wars: episode vi - return of the jedi',
                            'indiana jones and the last crusade',
                            'indiana jones and the temple of doom',
                            'jurassic park',
                            'the lost world: jurassic park',
                            'jurassic park iii',
                            'star trek v: the final frontier',
                            'star trek: the motion picture',
                            'star trek: first contact',
                            'star trek vi: the undiscovered country',
                            'star trek iv: the voyage home',
                            'the majestic',
                            'hannibal',
                            'star trek: insurrection',
                            'dr. strangelove or: how i learned to stop worrying and love the bomb'
                          ]),
          ('a nightmare on elm street', ['a nightmare on elm street part 2: freddy\'s revenge',
                                         'a nightmare on elm street: the dream child',
                                         'cruel intentions',
                                         'erin brockovich',
                                         'hellraiser: hellseeker',
                                         'little nicky',
                                        ]),]

To answer our question, we will now look at two ways of evaluating our rankings:
1. Precision vs. Recall plots

2. An evaluation statistic called Mean Average Precision

To start off, **complete the `precision_recall` function below.**
To recap, `precision @ K` measures the precision in the subset of documents that were retrieved _up to_ an index $K$, and `recall @ K` is defined similarly.

In [ ]:
def precision_recall(ranking_in, relevant):
    """
    Returns lists of precision and recall at different k values
    
    Parameters
    ----------
    ranking_in : str list 
        List with sorted ranking of movies (movie names), starting with the most similar, and ending
        with the least similar.
    relevant : str list
        List of movies (movie names) relevant to the original query

    Returns
    -------
    tuple: (np.ndarray, np.ndarray)
        Returns tuple such that tuple[0] is numpy array of precision at different k values and 
        tuple[1] is numpy array of recall at different k values. 
    
        tuple[0] -> precision: numpy array of length equal to the length+1 of ranking_in, where 
        precision[k] = the precision@k. Leave precision[0] to be 0.
        
        tuple[1] -> recall: numpy array of length equal to the length+1 of ranking_in, where 
        recall[k] = the recall@k. Leave recall[0] to be 0.
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that precision_recall returns the correct output"""
query, rel_movs = queries[1]
ranked_movs = [m for m,_ in get_ranked_movies(query, movie_sims_cos)]
precision, recall = precision_recall(ranked_movs, rel_movs)

assert precision[0] == 0
assert recall[0] == 0
assert precision.shape == (617,)
assert recall.shape == (617,)
assert sum(precision) > 48 and sum(precision) < 54
assert sum(recall) > 505 and sum(recall) < 511
assert precision[300] > 0.04 and precision[300] < 0.05
assert recall[300] > 0.8 and recall[300] < 0.9


## Plotting Precision-Recall Curves

Below, we have provided the code that uses matplotlib to create a recall (x-axis) vs. precision (y-axis) plot, plotting each of the 3 ground truth queries as seperate lines on the plot. For each query, we consider all N-1 movies other than the query itself when computing the ranking. The label of each line is the ground truth query. 

It's worth noting that considering precision/recall curves like this on the query level is a bit odd. In general, given more supervised data, one would compute aggregate precision/recall curves over a large number of ground truths. In this way, each line would represent a different information retrieval algorithm's performance over the test set of queries. However, we will be somewhat unorthodox here and plot one curve per query.

In [ ]:
def plot_precision_recall(matrix, xlim, ylim):
    """Plots the precision-recall curve given the similarity matrix
    
    Params: {matrix: np.ndarray,
             xlim: List,
             ylim: List}
    Returns: None
    """
    for query, rel_movs in queries:
        ranked_movs = [m for m,_ in get_ranked_movies(query, matrix)]
        precision, recall = precision_recall(ranked_movs, rel_movs)
        plt.plot(recall, precision)

    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.legend([q[0] for q in queries])

## Question 5b (Free Response): Cosine Similarity Plot Analysis

Run the code below to show the precision-recall curve for our three movie queries using cosine similarity. 

In [ ]:
# Cosine Similarity Plot
plot_precision_recall(movie_sims_cos, [0,1.1], [0,1.1])

Looking at the cosine similarity plot above, make some observations about why you're seeing what you're seeing. What does the plot reveal about the performance of the cosine similarity method for the given queries? Include observations on what worked well, what didn't, and how you've arrived at these conclusions. 

<br>


<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;"></div>

## Question 5c (Free Response): Jaccard Similarity Plot Analysis

Run the code below to show the precision-recall curve for our three movie queries using jaccard similarity. 

In [ ]:
# Jaccard Similarity Plot
plot_precision_recall(movie_sims_jac, [0,1.1], [0,0.4])

Looking at the jaccard similarity plot above,  make some observations about why you're seeing what you're seeing. What does the plot reveal about the performance of the jaccard similarity method for the given queries? Include observations on what worked well, what didn't, and how you've arrived at these conclusions. 

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;"></div>

## Question 5d (Free Response): Cosine Sim vs Jaccard Sim Plot Comparison

Now that you've observed the plots for both cosine similarity and jaccard similarity, please answer the following two questions:

1. Which system performs better, in general?
2. Which query was the most problematic in each case?

<br>

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;"></div>

## Question 5e (Code Completion): Calculating F-score Values

Now that you have seen the precison-recall, we will calculate the F-score foreach of the queries and see if they reveal any new information. As a reminder, the formuala for calculating the F-score is given below.

$$F-score = \frac{2}{\frac{1}{Precision} + \frac{1}{Recall}}$$ 

Note: If Precision or Recall is 0 we will just state that the fscore is 0.

In [ ]:
def compute_fscore(precision, recall):
    """
    Returns lists of f-score values at different k values, where fscore[k] = the fscore@k
    
    Parameters
    ----------
    precision : np.ndarray
        numpy array of precision values at different k values, where precision[k] = the
        precision@k.
    recall : np.ndarray
        numpy array of recall values at different k values, where recall[k] = the
        recall@k.

    Returns
    -------
    np.ndarray
        Returns a numpy array of length equal to the length of the precision (and recall) parameter 
        array, where fscore[k] = the fscore@k.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that precision_recall returns the correct output"""
query, rel_movs = queries[1]
ranked_movs = [m for m,_ in get_ranked_movies(query, movie_sims_cos)]
precision, recall = precision_recall(ranked_movs, rel_movs)
fscore = compute_fscore(precision, recall)

assert fscore[0] == 0
assert fscore.shape == (617,)
assert recall.shape == (617,)
assert sum(fscore) > 60 and sum(precision) < 100
assert fscore[300] > 0.08 and precision[300] < 0.09


In [ ]:
def plot_fscore(matrix):
    """Plots the fscore curve given the similarity matrix
    
    Params: {matrix: np.ndarray}
    Returns: None
    """
    query, rel_movs = queries[1]
    ranked_movs = [m for m,_ in get_ranked_movies(query, movie_sims_cos)]

    for query, rel_movs in queries:
        ranked_movs = [m for m,_ in get_ranked_movies(query, matrix)] 
        precision, recall = precision_recall(ranked_movs, rel_movs)
        fscore = compute_fscore(precision, recall)
        plt.plot(range(1,len(fscore)+1), fscore)

    plt.xlabel("K")
    plt.ylabel("Fscore")
    plt.legend([q[0] for q in queries])

In [ ]:
plot_fscore(movie_sims_cos)

In [ ]:
plot_fscore(movie_sims_jac)

## Question 5f (Free Response): Cosine Sim vs Jaccard Sim Fscore Plot Comparison

Now that you've observed the fscore plots for both cosine similarity and jaccard similarity, please answer the following three questions:

1. In what ways do the F-score curves provide more or different information about the retrievals compared to the precision-recall graphs?
2. Are there any queries for which the F-score curves suggest different rankings of movies than the precision-recall graphs? If so, which queries and what are the differences?
3. How would you interpret the shape of the F-score curves for a given query? What does a flat curve or a steeply increasing curve indicate about the rankings?

<br>

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;"></div>

## Evaluating our rankings with Mean Average Precision

While precision/recall curves are a good tool to visualize how well our rankings are performing, in some contexts, it is ideal to come up with a single number that characterizes how well our ranking system is doing over all ground-truth queries. A commonly used statistic for ranking systems is _Mean Average Precision_. To compute this value, first, one must compute the average precision for each query and subsequent ranking output by the system. Next, one averages these values. We've filled in functions for both `average_precision` and `mean_average_precision` for you. At the end of this section, run the provided code, which will give you a single number that evaluates the performance of each ranking system.

In [ ]:
def average_precision(ranking_in, relevant):
    '''
    Arguments:
        ranking_in: sorted ranking of movies, starting with the most most similar, and ending
        with the least similar.
        
        relevant: iterable of movies relevant to the original query
        
    Returns:
        average_precision: float corresponding to the AP statistic for this ranking and
        this set of relevant docuemnts.
    '''
    rel_rank = sorted([ranking_in.index(r)+1 for r in relevant])
    return np.mean([(i+1)*1./(r) for i, r in enumerate(rel_rank)])
    
    
def mean_average_precision(queries_in, sims_mat):
    '''
    Arguments:
        queries_in: a list of (query, [relevant documents]) pairs, representing the queries we
        want to evaluate our ranking system against
    
        sims_mat: a movie by movie numpy array, where sims_mat[i,j] = sims_mat[j,i] = the similarity
        of movies i and j
    
    Returns:
        mean_average_precision: float corresponding to the average AP statistic for the input queries
        and the similarity matrix
    '''
    rankings = [[movie_index_to_name[i] for i in np.argsort(sims_mat[movie_name_to_index[q],:])[::-1] if movie_index_to_name[i] != q]
                for q,_ in queries]
    aps = []
    for i, results in enumerate(rankings):
        ap = average_precision(results, queries_in[i][1])
        aps.append(ap)
        print("Query name:", queries_in[i][0], "//", "Avg. Precision: {:.3f}".format(ap))
    mean_avg_precision = 1.0 * sum(aps)/len(queries_in)
    print("[Mean Average Precision: {:.3f}]".format(mean_avg_precision))
    return mean_avg_precision

In [ ]:
print("=== cosine_sim with tfidf features ===")
mean_average_precision(queries, movie_sims_cos)
print()
print("=== jacc_sim ===")
mean_average_precision(queries, movie_sims_jac)
print()

## Rocchio Algorithm

Great -- we have a baseline evaluation for how well our vector space tfidf information retrevial system does! Now, it is our job to do something better. Here, we will implement the Rocchio Algorithm and use it for pseudo relevance feedback.

This is Rocchio’s query update rule for relevance feedback:

$$\overrightarrow{{q}_1} = a * \overrightarrow{{q}_0} + b*\frac{1}{|D_r|}\sum_{d \in D_r}{\overrightarrow{d}}-c*\frac{1}{|D_{nr}|}\sum_{d \in D_{nr}}{\overrightarrow{d}}$$ 

In the above, $\overrightarrow{{q}_0}$ is the initial query, $\overrightarrow{{q}_1}$ is the updated query, $D_r$ is the set of relevant documents and $D_{nr}$ is the set of non-relevant documents. $a$, $b$, and $c$ are the update weights that correspond to the original query, the relevant queries, and the irrelevant queries, respectively. If after the update, there are negative values in $\overrightarrow{{q}_1}$, they are set to 0.

## Question 6 (Code Completion): Rocchio
Your job here is to implement the Rocchio Algorithm for relevance feedback. This function will test your understanding of how numpy arrays interact with variable types. For example, a + b where a is a python float and b is a numpy array will return a numpy array where each element of b has been shifted by a. While this particular function may or may not be needed to implement rocchio, it can be *very* helpful. 

In [ ]:
def rocchio(query, relevant, irrelevant, input_doc_matrix, \
            movie_name_to_index,a=.3, b=.3, c=.8, clip = True):
    """Returns a vector representing the modified query vector. 
    
    Note: 
        If the `clip` parameter is set to True, the resulting vector should have 
        no negatve weights in it!
        
        Also, be sure to handle the cases where relevant and irrelevant are empty lists.
        
    Params: {query: String (the name of the movie being queried for),
             relevant: List (the names of relevant movies for query),
             irrelevant: List (the names of irrelevant movies for query),
             input_doc_matrix: Numpy Array,
             movie_name_to_index: Dict,
             a,b,c: floats (weighting of the original query, relevant queries,
                             and irrelevant queries, respectively),
             clip: Boolean (whether or not to clip all returned negative values to 0)}
    Returns: Numpy Array 
    """
    # YOUR CODE HERE
    raise NotImplementedError()

Before we use the Rocchio algorithm for understanding pseduo-relevance feedback, we'll do a reality check and make sure that our implementation can improve performance on our existing queries. To augment our analysis, movie guru Ilan has also provided you with a list of irrelevant queries for each of our test queries. You'll notice that the irrelevant queries for each of the test queries are quite similar. This is because it's much more common for a pair of movies to be irrelevant, rather than relevant. In fact, it's not uncommon that *all* queries that are not relevant are *assumed to be irrelevant.*

Here, though, we'll just use this small subset.

In [ ]:
irrelevant  = [('the matrix', ['lone star',
                               '2001: a space odyssey',
                               'wall street',
                               'the elephant man',
                               'eternal sunshine of the spotless mind',
                              'suburbia',
                               'taking sides',
                               'lake placid'
                              ]),
               ('star wars',  ['lone star',
                               '2001: a space odyssey',
                               'wall street',
                               'the elephant man',
                               'eternal sunshine of the spotless mind',
                              'suburbia',
                              'taking sides',
                              'lake placid']),
               ('a nightmare on elm street', ['lone star',
                               '2001: a space odyssey',
                               'wall street',
                               'the elephant man',
                               'eternal sunshine of the spotless mind',
                                'suburbia',
                                'taking sides',
                                'lake placid']),]


In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that rocchio returns the correct output"""
query_vector = rocchio("the matrix",[],irrelevant[0][1],doc_by_vocab,movie_name_to_index)
query_vector2 = rocchio("star wars",[],irrelevant[1][1],doc_by_vocab,movie_name_to_index)
assert type(query_vector) == np.ndarray
assert type(query_vector2) == np.ndarray
assert sum(query_vector) > 3 and sum(query_vector) < 4
assert sum(query_vector2) > 2 and sum(query_vector2) < 3

## Question 7 (Code Completion): Leveraging Rocchio
Your job is to leverage rocchio to return the top 10 highest ranked movies for each query, where the query vector is updated with rocchio. 
Return the dictionary with in the following format:
```
{'the matrix': [movie1,movie2,...,movie10],
 'star wars': [movie1,movie2,...,movie10],
 'a nightmare on elm street': [movie1,movie2,...,movie10]}
```


Note that the query itself should be excluded from the rankings.

In [ ]:
def top_10_with_rocchio(relevant_in, irrelevant_in, input_doc_matrix, \
            movie_name_to_index,movie_index_to_name,input_rocchio):
    """Returns a dictionary in the following format:
    {
        'the matrix': [movie1,movie2,...,movie10],
        'star wars': [movie1,movie2,...,movie10],
        'a nightmare on elm street': [movie1,movie2,...,movie10]
    }
    
    Note: 
        You can assume that relevant_in[i][0] = irrelevant_in[i][0] 
        (i.e. the queries are in the same order). 
        
        You should use the default rocchio parameters.
        
        You should NOT return the query itself in the list of most common
        movies.
        
    Parameters
    ----------
    relevant_in : (query: str, [relevant documents]: str list) list 
        List of tuples of the form:
        tuple[0] = name of movie being queried (str), 
        tuple[1] = list of names of the relevant movies to the movie being queried (str list).
    irrelevant_in : (query: str, [irrelevant documents]: str list) list 
        The same format as relevant_in except tuple[1] contains list of irrelevant movies instead.
    input_doc_matrix : np.ndarray
        The term document matrix of the movie transcripts. input_doc_mat[i][j] is the tfidf
        of the movie i for the word j.
    movie_name_to_index : dict
         A dictionary linking the movie name (Key: str) to the movie index (Value: int). 
         Ex: {'movie_0': 0, 'movie_1': 1, .......}
    movie_index_to_name : dict
         A dictionary linking the movie index (Key: int) to the movie name (Value: str). 
         Ex: {0:'movie_0', 1:'movie_1', .......}
    input_rocchio: function
        A function implementing the rocchio algorithm. Refer to Q6 for the function 
        input parameters. Make sure you use the function's default parameters as 
        much as possible.
        
    Returns
    -------
    dict
        Returns the top ten highest ranked movies for each query in the format described above.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
movie_recommend_with_rocchio = top_10_with_rocchio(queries, irrelevant,\
                                                  doc_by_vocab,movie_name_to_index,\
                                                  movie_index_to_name,rocchio)
for k,v in movie_recommend_with_rocchio.items():
    print(k)
    print("="*len(k))
    [print(a) for a in v]
    print("")

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that top_10_with_rocchio returns the correct output"""
assert type(movie_recommend_with_rocchio) == dict
assert "the x files" in movie_recommend_with_rocchio["the matrix"]
assert "star wars: the empire strikes back" in movie_recommend_with_rocchio["star wars"]
assert "a nightmare on elm street 3: dream warriors" in \
    movie_recommend_with_rocchio["a nightmare on elm street"]

## Question 7b (Code Completion): Leveraging Rocchio
Your job is to implement `mean_average_precision_rocchio` below, to compute MAP using the Rocchio-updated query vectors.

Your function should call `rocchio` and `average_precision`.

In [ ]:
def mean_average_precision_rocchio(relevant_in, irrelevant_in, input_doc_matrix, \
            movie_name_to_index, movie_index_to_name, input_rocchio):
    """Returns a float corresponding to the mean AP statistic for the Rocchio-updated input queries
        and the similarity matrix
    Note: 
        You can assume that relevant_in[i][0] = irrelevant_in[i][0] 
        (i.e. the queries are in the same order). 
        
        You should use the default rocchio parameters.
        
        You should NOT include the query itself in the list of most common
        movies.
        
    Parameters
    ----------
    relevant_in : (query: str, [relevant documents]: str list) list 
        List of tuples of the form:
        tuple[0] = name of movie being queried (str), 
        tuple[1] = list of names of the relevant movies to the movie being queried (str list).
    irrelevant_in : (query: str, [irrelevant documents]: str list) list 
        The same format as relevant_in except tuple[1] contains list of irrelevant movies instead.
    input_doc_matrix : np.ndarray
        The term document matrix of the movie transcripts. input_doc_mat[i][j] is the tfidf
        of the movie i for the word j.
    movie_name_to_index : dict
         A dictionary linking the movie name (Key: str) to the movie index (Value: int). 
         Ex: {'movie_0': 0, 'movie_1': 1, .......}
    movie_index_to_name : dict
         A dictionary linking the movie index (Key: int) to the movie name (Value: str). 
         Ex: {0:'movie_0', 1:'movie_1', .......}
    input_rocchio: function
        A function implementing the rocchio algorithm. Refer to Q6 for the function 
        input parameters. Make sure you use the function's default parameters as 
        much as possible.
        
    Returns
    -------
    float
        Returns a float corresponding to the mean AP statistic for the Rocchio-updated input queries
        and the similarity matrix
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This is an autograder test. Here we can test the function you just wrote above.
"""Check that mean_average_precision_rocchio returns the correct output"""
mean_average_precision = mean_average_precision_rocchio(queries, irrelevant,\
                                                  doc_by_vocab,movie_name_to_index,\
                                                  movie_index_to_name,rocchio)
assert mean_average_precision > 0.60

The Rocchio addition to the system clearly adds a much larger level of precision as you are doing query
modification that is based on relevant and irrelevant documents that are passed in. With the revision
of this query this increases the search engine's recall, as well as the precision.

**Note:**
While our performance increased, it shouldn't be surprising that this is the case. At a high level, here is what we did: we took our original query vectors and moved them slightly closer to the "ground truth" relevant documents, and then used mean average precision to find that, indeed, the new query vector was closer to the true "ground truth" vectors. 

## Question 8 (Free Response): Biggest Losses

However, it is fun to see what words were given more or less weight for a given query according to the Rocchio modified query vector. Understand and run the following code:

In [ ]:
q_o = doc_by_vocab[movie_name_to_index[queries[0][0]],:]
q_m = rocchio(queries[0][0], queries[0][1], irrelevant[0][1],doc_by_vocab,movie_name_to_index)
diffs = q_m-q_o
args_diffs = np.argsort(diffs)
losses = args_diffs[:10]
print("Biggest losses:")
for l in losses:
    print("{}:{:.3f}".format(index_to_vocab[l], diffs[l]))

In the cell below, explain what the code is doing and what the findings show

<br>

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;">Write your answer in the provided cell below</div>

YOUR ANSWER HERE

<div style="border-bottom: 4px solid #AAA; padding-bottom: 6px; font-size: 16px; font-weight: bold;"></div>